In [ ]:
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [25, 15]
import itertools
from enum import Enum

print(1 + (3 - 1) % 10)

In [ ]:
def to_grayscale( img: np.ndarray ):
    return 0.3 * img[:, :, 0] + 0.59 * img[:, :, 1] + 0.11 * img[:, :, 2]

In [ ]:
Kx = np.array([
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]
], float)

Ky = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
], float)


In [ ]:
def apply_to_patch(ker: np.ndarray, a2: np.ndarray):
    assert(ker.shape[:2] == a2.shape[:2])
    if len( a2.shape ) == 2:
        a2 = np.expand_dims(a2, -1)

    idx = (ker != 0)

    if a2.shape[-1] == 1:
        return ker[idx].dot( a2[idx, 0] )
    else:
        ret = np.stack([
            ker[idx].dot( a2[idx, i] )
            for i in range(a2.shape[-1])
        ], -1)
        return ret

def convolve(kernel: np.ndarray, img: np.ndarray):
    ksz = kernel.shape[0]
    halfksz = int(ksz / 2)
    padding = [(halfksz, halfksz), (halfksz, halfksz)]
    if len(img.shape) == 3:
        padding += [(0,0)]
    padded_img = np.pad(img, padding , 'reflect')

    ret = np.zeros_like(img)
    for i, j in itertools.product(range( img.shape[0] ), range( img.shape[1] )):
        ip = i + halfksz
        jp = j + halfksz

        l = ip - halfksz
        r = ip + halfksz
        u = jp - halfksz
        d = jp + halfksz

        ret[i, j] = apply_to_patch(kernel, padded_img[l:r + 1, u:d + 1])

    return ret

In [ ]:
img = np.array(PIL.Image.open( 'data/0018.jpg' ))
img = to_grayscale(img)

dx = convolve(Kx, img)
dy = convolve(Ky, img)
grad = np.sqrt(dx**2 + dy**2)
grad = 255 * grad / np.max(grad)

plt.subplot( 2, 2, 1 )
plt.title('Original')
plt.imshow( img, cmap='gray' )

plt.subplot( 2, 2, 2 )
plt.title('dx')
plt.imshow( dx, cmap='gray' )

plt.subplot( 2, 2, 3 )
plt.title('dy')
plt.imshow( dy, cmap='gray' )

plt.subplot( 2, 2, 4 )
plt.title('G')
plt.imshow( grad, cmap='gray' )

plt.show()

plt.imshow( grad > 35, cmap='gray' )
plt.show()